In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [65]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [66]:
torch.cuda.is_available()

True

In [67]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = get_default_device()
device

device(type='cuda')

In [68]:
def to_device(data,device):
    if isinstance(data, (list,tuple)): #The isinstance() function returns True if the specified object is of the specified type, otherwise False.
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking=True)

In [69]:
class DeviceDataLoader():
    def __init__(self,dl,device):
        self.dl = dl
        self.device = device
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b,self.device)
    
    def __len__(self):
        return len(self.dl)

In [70]:
# Define the CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output

# Load MNIST dataset
train_dataset = datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

test_dataset = datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
train_loader = DeviceDataLoader(train_loader, device)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=True)
test_loader = DeviceDataLoader(test_loader, device)
    
# Initialize the model
model = Net().to(device)

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the model
for epoch in range(15):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{}]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dl.dataset), loss))

c:\Users\karin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 0 [0/60000]	Loss: 2.305815
Train Epoch: 0 [6400/60000]	Loss: 0.190469
Train Epoch: 0 [12800/60000]	Loss: 0.269778
Train Epoch: 0 [19200/60000]	Loss: 0.107482
Train Epoch: 0 [25600/60000]	Loss: 0.081234
Train Epoch: 0 [32000/60000]	Loss: 0.139888
Train Epoch: 0 [38400/60000]	Loss: 0.278301
Train Epoch: 0 [44800/60000]	Loss: 0.229512
Train Epoch: 0 [51200/60000]	Loss: 0.103200
Train Epoch: 0 [57600/60000]	Loss: 0.153382
Train Epoch: 1 [0/60000]	Loss: 0.110708
Train Epoch: 1 [6400/60000]	Loss: 0.148999
Train Epoch: 1 [12800/60000]	Loss: 0.079755
Train Epoch: 1 [19200/60000]	Loss: 0.184182
Train Epoch: 1 [25600/60000]	Loss: 0.050171
Train Epoch: 1 [32000/60000]	Loss: 0.077771
Train Epoch: 1 [38400/60000]	Loss: 0.010460
Train Epoch: 1 [44800/60000]	Loss: 0.107510
Train Epoch: 1 [51200/60000]	Loss: 0.021701
Train Epoch: 1 [57600/60000]	Loss: 0.083223
Train Epoch: 2 [0/60000]	Loss: 0.097227
Train Epoch: 2 [6400/60000]	Loss: 0.022836
Train Epoch: 2 [12800/60000]	Loss: 0.078335
Tra

In [71]:
# Evaluate the model
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += loss_function(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

print('Test loss:', test_loss)
print('Test accuracy:', correct / len(test_loader.dl.dataset))

Test loss: 0.3200238863937557
Test accuracy: 0.9922


In [72]:
# Save the model
torch.save(model.state_dict(), 'mnist_cnn.pth')